In [2]:
require(TSDecomposition)
require(foreach)
require(parallel)
require(doMC)
require(compiler)
require(Rssa)
require(FNN)

source('utils.r')

In [3]:
setCompilerOptions(suppressUndefined=TRUE)
null=compilePKGS(enable=TRUE)
null=enableJIT(3)
setCompilerOptions(optimize=3)
cores = detectCores(all.tests = FALSE, logical = TRUE)

In [4]:
seriesFolder = 'data'
modelFolder  = 'model'
resultFolder = 'testResult'

In [5]:
#select all series file in the data folder
seriesFile = list.files(path = seriesFolder, pattern = NULL, all.files = FALSE, full.names = TRUE, recursive = FALSE)

---
Fourier Decompostion
===

Computes the Discrete Fourier Transform (DFT) of an array with a fast algorithm, the “Fast Fourier Transform” (FFT). Uses C translation of Fortran code in Singleton (1979).

When z is a vector, the value computed and returned by fft is the unnormalized univariate discrete Fourier transform of the sequence of values in z. Specifically, y <- fft(z) returns

$$y[h] = \sum_{k=1}^n z[k]*exp{(\frac{-2*\pi*1i*(k-1)*(h-1)}{n})}$$

for $h = 1, ..., n$ where $n = length(y)$. If inverse is TRUE, $exp(-2*pi...)$ is replaced with $exp(2*pi...)$. When z contains an array, fft computes and returns the multivariate (spatial) transform. If inverse is TRUE, the (unnormalized) inverse Fourier transform is returned, i.e., if y <- fft(z), then z is fft(y, inverse = TRUE) / length(y). By contrast, mvfft takes a real or complex matrix as argument, and returns a similar shaped matrix, but with each column replaced by its discrete Fourier transform. This is useful for analyzing vector-valued series. The FFT is fastest when the length of the series being transformed is highly composite (i.e., has many factors). If this is not the case, the transform may take a long time to compute and will use a large amount of memory.

**References**

Becker, R. A., Chambers, J. M. and Wilks, A. R. (1988) The New S Language. Wadsworth & Brooks/Cole.

Singleton, R. C. (1979) Mixed Radix Fast Fourier Transforms, in Programs for Digital Signal Processing, IEEE Digital Signal Processing Committee eds. IEEE Press.

Cooley, James W., and Tukey, John W. (1965) An algorithm for the machine calculation of complex Fourier series, Math. Comput. 19(90), 297–301. https://dx.doi.org/10.1090/S0025-5718-1965-0178586-1

In [9]:
fourierDec <- function(seriesObj, par){

    freq.cutoff = unlist(par[1])
    series      = seriesObj$series

    freq.cutoff = freq.cutoff
    coeffs = fft(series)
    mags   = coeffs[1:(length(coeffs)/2)]
    mags   = 1+sqrt(Re(mags)^2+Im(mags)^2)
    o.idx  = order(mags, decreasing = T)
    idx    = (1:length(mags))[-o.idx[1:freq.cutoff]]
        
    coeffs[idx] = complex(real=0, imaginary=0)
    coeffs[length(coeffs) - idx + 1] = complex(real=0, imaginary=0)
    det = Re(fft(coeffs, inverse=T)) / length(series)
    
    return(det)
}

In [10]:
#all combination of possible parameters for Fourier Algorithm
params = expand.grid(
  cutoff = 1:500,
  stringsAsFactors = FALSE
)

In [11]:
resultTable = gridSearch(fourierDec, params, seriesFile, 'Fourier', cores)

In [12]:
write.csv(resultTable, file=paste(resultFolder,'/fourier.csv', sep=''), row.names=FALSE)

---
Wavelets Decompostion
===

| Parameter |                                                                                                                                                                                                                                                                                                       |
|-----------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  filters  | Either a wt.filter object, a character string indicating which wavelet filter to use in the decomposition, or a numeric vector of wavelet coefficients (not scaling coefficients). See above for acceptable filter names.                                                                   |
|  n.levels | An integer specifying the level of the decomposition. By default this is the value J such that the length of the series is at least as great as the length of the level J wavelet filter, but less than the length of the level J+1 wavelet filter. Thus, j <= log((N-1)/(L-1)+1), where N is the length of the series. |
|  boundary | A character string indicating which boundary method to use. boundary = "periodic" and boundary = "reflection" are the only supported methods at this time.                                                                                                                                            |


**Filter Details**

The character strings currently supported are derived from one of four classes of wavelet transform filters: Daubechies, Least Asymetric, Best Localized and Coiflet. The prefixes for filters of these classes are d, la, bl and c, respectively. Following the prefix, the filter name consists of an integer indicating length. Supported lengths are as follows:

* Daubechies: 2,4,6,8,10,12,14,16,18,20.
* Least Asymetric: 8,10,12,14,16,18,20.
* Best Localized: 14,18,20.
* Coiflet: 6,12,18,24,30.

Thus, to obtain the Daubechies wavelet transform filter of length 4, the character string "d4" can be passed to wt.filter. 

This naming convention has one exception: the Daubechies wavelet transform filter of length 2 is denoted by haar instead of d2.

In [13]:
waveletDec.script <- function(seriesObj, par){

    filter   = unlist(par[1])
    n.levels = unlist(par[2])
    boundary = unlist(par[3])
    series   = seriesObj$series

    r.wavelet = wavelets::dwt(series,
                            filter = filter,
                            n.levels=n.levels,
                            boundary=boundary,
                            fast=TRUE)
    for (i in 1:length(r.wavelet@W)) {
        r.wavelet@W[[i]] = cbind(rep(0, length(r.wavelet@W[[i]])))
    }
    det = wavelets::idwt(r.wavelet)
    return(det)
}
waveletDec = cmpfun(waveletDec.script)

In [14]:
#all combination of possible parameters for wavelets Algorithm
params = expand.grid(
  filters = c("haar", "d4", "d6", "d8", "d10", "d12", "d14", "d16", "d18", "d20",#Daubechies
              "la8", "la10", "la12", "la14", "la16", "la18", "la20", #Least Asymetric
              "bl14", "bl18", "bl20", #Best Localized
              "c6", "c12", "c18", "c24", "c30"), #Coiflet
  n.levels = 1:50,
  boundarys = c("periodic","reflection"),
  stringsAsFactors = FALSE
)

In [15]:
resultTable = gridSearch(waveletDec, params, seriesFile, 'Wavelets', cores)

In [16]:
write.csv(resultTable, file=paste(resultFolder,'/wavelets.csv', sep=''))

---
SSA Decomposition
===

| Parameter |                                                                                                                                                                                                                                                                    |
|-----------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|     L     | integer, window length. Fixed to half of the series length by default. Should be vector of length 2 for 2d SSA                                                                                                                                                     |
|    neig   | integer, number of desired eigentriples. If 'NULL', then sane default value will be used, see 'Details'                                                                                                                                                            |
|    kind   | SSA method. This includes ordinary 1d SSA, 2d SSA, Toeplitz variant of 1d SSA, multichannel variant of SSA and complex SSA                                                                                                                                         |
| circular  | logical vector of one or two elements, describes series topology for 1d SSA and Toeplitz SSA or field topology for 2d SSA. 'TRUE' means series circularity for 1d case or circularity by a corresponding coordinate for 2d case. See (Shlemov, 2014) for more info |

**Variants of SSA**

The following implementations of the SSA method are supported (corresponds to different values of kind argument):

* **1d-ssa**: Basic 1d SSA as described in Chapter 1 of (Golyandina et al, 2001). This is also known as Broomhead-King variant of SSA or BK-SSA, see (Broomhead and King, 1986).
* **toeplitz-ssa**: Toeplitz variant of 1d SSA. See Section 1.7.2 in (Golyandina et al, 2001). This is also knows as Vatuard-Gill variant of SSA or VG-SSA for analysis of stationary time series, see (Vautard and Ghil, 1989).
* **mssa**: Multichannel SSA for simultaneous decomposition of several time series (possible of unequal length). See (Golyandina and Stepanov, 2005).
* **cssa**: Complex variant of 1d SSA.
* **2d-ssa**: 2d SSA for decomposition of images and arrays. See (Golyandina and Usevich, 2009, and Golyandina et.al, 2015) for more information.
* **nd-ssa**: Multidimensional SSA decomposition for arrays (tensors).

**References**

Broomhead, D.S., and King, G.P. (1986a): Extracting qualitative dynamics from experimental data, Physica D, 20, 217–236.

Vautard, R., and Ghil, M. (1989): Singular spectrum analysis in nonlinear dynamics, with applications to paleoclimatic time series, Physica D, 35, 395–424.

Golyandina, N., Nekrutkin, V. and Zhigljavsky, A. (2001): Analysis of Time Series Structure: SSA and related techniques. Chapman and Hall/CRC. ISBN 1584881941

Golyandina, N. and Stepanov, D. (2005): SSA-based approaches to analysis and forecast of multidimensional time series. In Proceedings of the 5th St.Petersburg Workshop on Simulation, June 26-July 2, 2005, St. Petersburg State University, St. Petersburg, 293–298. http://www.gistatgroup.com/gus/mssa2.pdf

Golyandina, N. and Usevich, K. (2009): 2D-extensions of singular spectrum analysis: algorithm and elements of theory. In Matrix Methods: Theory, Algorithms, Applications. World Scientific Publishing, 450-474.

In [17]:
ssaDec.script <- function(seriesObj, par){

    L        = unlist(par[1])
    neig     = unlist(par[2])
    kind     = unlist(par[3])
    circular = unlist(par[4])
    series   = seriesObj$series

    #execute
    s = ssa(series, L=L, neig=neig, kind=kind, circular=circular)
    r = reconstruct(s, groups = seq(1:(L/2)))

    #mutual information to separate deterministic components
    mi = c()
    for(i in 1:(length(r)-1)) 
        mi = c(mi, FNN::mutinfo(r[[i]], r[[i+1]]))
    
    det.idx = which.max(abs(diff(mi))) + 1
    
    #sum det. comp.
    detComp = rep(0, length(r[[1]]))
    for(i in 1:det.idx) 
        detComp = detComp + r[[i]]

    return(detComp)
}
ssaDec = cmpfun(ssaDec.script)

In [18]:
#all combination of possible parameters for wavelets Algorithm
params = expand.grid(
  L = 1:50,
  neig = 1:50,
  kind = c('1d-ssa', 'toeplitz-ssa', 'mssa', 'cssa', 'cssa', '2d-ssa', 'nd-ssa'),
  circular = c(TRUE, FALSE),
  stringsAsFactors = FALSE
)

In [19]:
resultTable = gridSearch(ssaDec, params, seriesFile, 'SSA', cores)

In [20]:
write.csv(resultTable, file=paste(resultFolder,'/ssa.csv', sep=''))

---
EMD-RP Decomposition
===

Function used to decompose a time series into 2 components: one stochastic and another deterministic. This function uses Empirical Mode Decomposition and Recurrence Plot functions. For more details about this method see: (Rios and Mello, 2013) For more details about RP: (Eckmann et al., 1987; Marwan et al., 2007) For more detail about EMD: (Huang et al., 1998)

| Parameter |                                                                      |
|-----------|----------------------------------------------------------------------|
|  detlevel | determinism threshold [0,1]                                          |
|   thresh  | epsilon chosen to determine the recurrent points - estimated if null |
|   delay   | delay dimension (phase space) - estimated if null                    |
| embedded  | embedded dimension (phase space) - estimated if null                 |

**References**

Ricardo Araújo Rios, Rodrigo Fernandes de Mello (2013): Improving time series modeling by decomposing and analyzing stochastic and deterministic influences, Signal Processing, Volume 93, Issue 11, November 2013, Pages 3001-3013, ISSN 0165-1684, http://dx.doi.org/10.1016/j.sigpro.2013.04.017.

Norden E. Huang, Zheng Shen, Steven R. Long, Manli C. Wu, Hsing H. Shih, Quanan Zheng, Nai-Chyuan Yen, Chi Chao Tung, Henry H. Liu (1998): The empirical mode decomposition and the Hilbert spectrum for nonlinear and non-stationary time series analysis - Proc. R. Soc. Lond. A 1998 454 903-995; DOI: 10.1098/rspa.1998.0193. Published 8 March 1998

In [21]:
emdrpDec   <- function(seriesObj, params){
  detlevel = unlist(params[1])
  thresh   = unlist(params[2])
  delay    = seriesObj$det.sepDim + unlist(params[3])
  embedded = seriesObj$det.embDim + unlist(params[4])
  series   = seriesObj$series
  emdrp    = rpemdDecomposition(series, detlevel, thresh, delay, embedded)
  return(emdrp@deterministic)
}

In [5]:
params = expand.grid(
  detlevel = seq(0.01, 0.99, by=0.01),
  thresh   = seq(0.01, 0.99, by=0.01),
  delay    = 0:4,
  embedded = 0:4,
  stringsAsFactors = FALSE
)
nrow(params)

[1] 245025

In [ ]:
resultTable = gridSearch(emdrpDec, params, seriesFile, 'EMDRP', cores)

In [ ]:
write.csv(resultTable, file=paste(resultFolder,'/emdrp.csv', sep=''))

---
EMD-MI Decomposition
===

Ricardo Araújo Rios, Rodrigo Fernandes de Mello(2016): Applying Empirical Mode Decomposition and mutual information to separate stochastic and deterministic influences embedded in signals, Signal Processing, Volume 118, January 2016, Pages 159-176, ISSN 0165-1684, http://dx.doi.org/10.1016/j.sigpro.2015.07.003.

In [31]:
emdmiDec <- function(seriesObj, par){
    series = seriesObj$series
    r.emd  = emd(series)
    mags   = c();
    for(i in 1 : r.emd$nimf){
        fft  = fft(r.emd$imf[,i])
        mags = cbind(mags,atan(Im(fft)/Re(fft)))
    }
    fft  = fft(r.emd$residue)
    mags = cbind(mags,atan(Im(fft)/Re(fft)))

    mi = c()
    for(i in 1 : r.emd$nimf){
        mi = c(mi, FNN::mutinfo(mags[,i], mags[,i+1]))
    }

    l   = which.max(abs(diff(mi))) + 1
    idx = l:r.emd$nimf
    det = ifelse(length(idx) != 1, rowSums(r.emd$imf[,idx]), r.emd$imf[,idx]) + r.emd$residue
    return(det)
}

In [32]:
params = expand.grid(none = TRUE)

In [33]:
resultTable = gridSearch(emdmiDec, params, seriesFile, 'EMDMI', cores)

In [34]:
write.csv(resultTable, file=paste(resultFolder,'/emdmi.csv', sep=''))

---
ForceDec
===

| Parameter |                                                                                                                                                                                                                       |
|-----------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|     k     | number of nearest neighbor to be considered                                                                                                                                                                           |
|   num.it  | maximum number of iteration                                                                                                                                                                                           |
|  epsilon  | minimum average displacement used to early stop ForceDec. The premise used here is that once the state movimentation stops, the atractor converged to the topology of the deterministic component in the phase-space. |
|   delta   | the percentage of the i-th iteration position will be preserved for each state in the phase-space                                                                                                                     |
|   delay   | delay dimension used to emerge the time series in the phase space.                                                                                                                                                    |
|  embedded | embedded dimension used to emerge the time series in the phase space.                                                                                                                                                 |

For evaluate all ForceDec possibility we set up epsilon (which is used to stop the algorithm iterations based on the displacement) with small number so we can reach a high number of iterations. Also, we modified the algorithm to calculate the MDA and MDDL after each iteration. This modification was done inded to accelarate the test once restart the algorithm to recalculate the metrics for diferent iteration number is a unecessary computational effort.

The original code without modification to tune test is following:

```R
forceDec   <- function(seriesObj, params){
  k        = params[1]
  num.it   = params[2]
  epsilon  = params[3]
  delta    = params[4]
  delay    = seriesObj$det.sepDim + params[5]
  embedded = seriesObj$det.embDim + params[6]
  series   = seriesObj$series

  s.emb = embedd(series, m=embedded, d=delay)
  pos.i = s.emb
  nn = get.knn(s.emb, k=k+1)$nn.index #search for k-nearest neighbor
  nn = cbind(1:nrow(nn), nn) #place itself as a neighbor
  norm.simil = c()
  for (i in 1:nrow(nn)){
      neigh.pos  = s.emb[nn[i,],]
      neigh.dist = as.matrix(dist(neigh.pos))[1,]
      neigh.dist[which(neigh.dist < epsilon)] = 0
      similarity = 1 - neigh.dist/(max(neigh.dist)+epsilon)
      similarity[which(similarity < epsilon)] = 0
      norm.simil = rbind(norm.simil, similarity[-1]/(sum(similarity[-1])))
  }

  disp.vec = c()
  it = -1
  for(it in 1:num.it){ #this loop can be done parallel to accelerate processing
    disp = 0
    for (i in 1:nrow(nn)){
      neigh.pos  = s.emb[nn[i,],]
      pos.i[i,]  = ((  delta   * neigh.pos[1,])
                 + ((1-delta)  * colSums(neigh.pos[-1,]*norm.simil[i,])))
      disp = disp + dist(rbind(pos.i[i,], s.emb[i,]))
    }
    disp = disp/nrow(nn)
    disp.vec = c(disp.vec, disp)
    if(disp <= epsilon) break;
    s.emb = pos.i
  }

  return(list(series=s.emb[,1], num.it=it))
}
```

In [6]:
forceDec   <- function(seriesObj, par, test.execution=TRUE, detComp=NULL){
    k        = unlist(par[1])
    num.it   = unlist(par[2])
    epsilon  = unlist(par[3])
    delta    = unlist(par[4])
    delay    = unlist(seriesObj$det.sepDim + par[5])
    embedded = unlist(seriesObj$det.embDim + par[6])
    series   = unlist(seriesObj$series)

    mddl.results = c()
    mda.results = c()

    s.emb = embedd(seriesObj$series, m=embedded, d=delay)
    pos.i = s.emb
    nn = get.knn(s.emb, k=k+1)$nn.index #search for k-nearest neighbor
    nn = cbind(1:nrow(nn), nn) #place itself as a neighbor
    norm.simil = c()
    for (i in 1:nrow(nn)){
        neigh.pos  = s.emb[nn[i,],]
        neigh.dist = as.matrix(dist(neigh.pos))[1,]
        neigh.dist[which(neigh.dist < epsilon)] = 0
        similarity = 1 - neigh.dist/(max(neigh.dist)+epsilon)
        similarity[which(similarity < epsilon)] = 0
        norm.simil = rbind(norm.simil, similarity[-1]/(sum(similarity[-1])))
    }

    disp.vec = c()
    it = -1
    #this loop can be done parallel to accelerate processing
    for(it in 1:num.it){ 
        disp = 0
        for (i in 1:nrow(nn)){
            neigh.pos  = s.emb[nn[i,],]
            pos.i[i,]  = ((  delta   * neigh.pos[1,])
                       + ((1-delta)  * colSums(neigh.pos[-1,]*norm.simil[i,])))
            disp = disp + dist(rbind(pos.i[i,], s.emb[i,]))
        }
        disp = disp/nrow(nn)
        disp.vec = c(disp.vec, disp)
        #if(disp <= epsilon) break;
        s.emb = pos.i

        #--------------------------------------------------------------
        # This evaluation is running embedded in the forceDec function
        # in order to accelerate the test, indeed this code does not
        # belong to this function and must not be implemented in real
        # scenarios
        #--------------------------------------------------------------
        if(test.execution){
            mddl.r = mddl(seriesObj$det.series, s.emb[,1])
            mda.r  = mda (seriesObj$det.series, s.emb[,1], delay, embedded)

            mddl.results = c(mddl.results, mddl.r)
            mda.results  = c(mda.results,  mda.r)
        }
        #--------------------------------------------------------------
        # Evaluation code end.
        #--------------------------------------------------------------
    }

    return(list(mda = mda.results, mddl = mddl.results))
}

In [7]:
params = expand.grid(
  k = c(2:10,15,20,25,30,25,40,45,50),
  num.it = 30,
  epsilon = 10^-6,
  delta = c(0.01, seq(0.05, 1, by=0.05)),
  delay = 0:4,
  embedded = 0:4
)
nrow(params)

[1] 8925

In [ ]:
resultTable = gridSearch(forceDec, params, seriesFile, 'ForceDec', cores)

In [25]:
write.csv(resultTable, file=paste(resultFolder,'/emdrp.csv', sep=''))